In [3]:
import cv2

def get_frame_count(video_path):
    """
    Get the total number of frames in a video file.
    
    Args:
        video_path (str): Path to the video file
    
    Returns:
        int: Total number of frames, or -1 if error
    """
    try:
        # Open the video file
        cap = cv2.VideoCapture(video_path)
        
        if not cap.isOpened():
            print("Error: Could not open video file")
            return -1
        
        # Get the total frame count
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        # Release the video capture object
        cap.release()
        
        return frame_count
    
    except Exception as e:
        print(f"Error: {e}")
        return -1

# Your video path
video_path = r"C:\Users\Admin\Documents\Akseli\bird_video_audio\output.avi"

# Get frame count
num_frames = get_frame_count(video_path)

if num_frames != -1:
    print(f"Total number of frames: {num_frames}")
    
    # Optional: Get additional video information
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    duration = num_frames / fps if fps > 0 else 0
    cap.release()
    
    print(f"FPS: {fps:.2f}")
    print(f"Duration: {duration:.2f} seconds")
else:
    print("Failed to get frame count")

Total number of frames: 1386
FPS: 5.00
Duration: 277.20 seconds


In [ ]:
from audioio import play, load_audio